In [ ]:
!pip install flask pyngrok nltk scikit-learn joblib -q

In [ ]:
# Imports
import pandas as pd
import string
import nltk
import joblib
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import os
import threading

# Download stopwords
nltk.download('stopwords', quiet=True)

# Define text cleaning function
def clean_text_for_prediction(text_input):
    text_input = str(text_input).lower()
    text_input = ''.join([c for c in text_input if c not in string.punctuation])
    tokens = text_input.split()
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    return ' '.join(tokens)

# Load model and vectorizer
def load_model_and_vectorizer():
    global loaded_model, loaded_vectorizer
    model_path = '/content/news_model.pkl'
    vectorizer_path = '/content/vectorizer.pkl'

    if not os.path.exists(model_path) or not os.path.exists(vectorizer_path):
        print("Model or vectorizer file not found at specified paths.")
        try:
            from google.colab import files
            print("Please upload 'news_model.pkl' and 'vectorizer.pkl'.")
            uploaded = files.upload()
        except ImportError:
            print("Not in Colab. Ensure files are at /content/.")
            return False

    try:
        loaded_model = joblib.load(model_path)
        loaded_vectorizer = joblib.load(vectorizer_path)
        print("✅ Model and vectorizer loaded successfully.")
        return True
    except Exception as e:
        print(f"❌ Error loading model/vectorizer: {e}")
        return False

# Create Flask app
app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>News Authenticity Checker</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background-color: #f4f4f9; }
        h1 { text-align: center; color: #333; }
        .container { max-width: 600px; margin: 0 auto; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 0 10px rgba(0,0,0,0.1); }
        textarea { width: 100%; height: 100px; margin: 10px 0; padding: 10px; }
        input[type=submit] { background-color: #4CAF50; color: white; padding: 10px 20px; border: none; border-radius: 4px; cursor: pointer; }
        input[type=submit]:hover { background-color: #45a049; }
        .result { margin-top: 20px; font-weight: bold; color: #333; }
    </style>
</head>
<body>
    <div class="container">
        <h1>📰 News Authenticity Checker</h1>
        <form method="POST">
            <textarea name="news_text" placeholder="Enter news text here">{{ S_TEXT if S_TEXT else '' }}</textarea>
            <input type="submit" value="Check Authenticity">
        </form>
        {% if PRED_RESULT %}
        <div class="result">Prediction: {{ PRED_RESULT }}</div>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def home():
    print("Received request at /")
    prediction_result = None
    user_input = None
    if request.method == "POST":
        print("Processing POST request")
        news_text = request.form.get("news_text", "").strip()
        user_input = news_text
        if news_text:
            try:
                cleaned_text = clean_text_for_prediction(news_text)
                vectorized_input = loaded_vectorizer.transform([cleaned_text])
                pred = loaded_model.predict(vectorized_input)[0]
                prediction_result = "REAL" if pred == 1 else "FAKE"
                print(f"Prediction completed: {prediction_result}")
            except Exception as e:
                prediction_result = f"Error: {e}"
                print(f"Prediction error: {e}")
    return render_template_string(html_template, PRED_RESULT=prediction_result, S_TEXT=user_input)

# Function to run Flask app
def run_flask():
    app.run(host="0.0.0.0", port=5000)

# Main execution
if __name__ == "__main__":
    print("🔁 Starting Execution...")
    if load_model_and_vectorizer():
        # Set ngrok auth token
        NGROK_AUTH_TOKEN = "2xV3q6f39efPkbaVMQ53H5TMafv_5ZckYvEzZnMubgauwFnv9"
        try:
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            print("✅ Ngrok auth token set successfully.")
        except Exception as e:
            print(f"❌ Error setting ngrok auth token: {e}")
            exit(1)

        try:
            ngrok.kill()  # Clean any old tunnels
            print("✅ Cleared old ngrok tunnels.")
            public_url = ngrok.connect(5000, bind_tls=True)
            print(f"🌐 App is running at: {public_url}")

            # Start Flask in a separate thread to avoid blocking
            flask_thread = threading.Thread(target=run_flask)
            flask_thread.start()
            flask_thread.join()  # Wait for the Flask app to finish
        except Exception as e:
            print(f"❌ Could not launch app: {e}")
    else:
        print("❌ Could not load model or vectorizer. Please check file names.")

🔁 Starting Execution...
✅ Model and vectorizer loaded successfully.
✅ Ngrok auth token set successfully.
✅ Cleared old ngrok tunnels.
🌐 App is running at: NgrokTunnel: "https://4b32-34-16-255-200.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/May/2025 14:56:06] "GET / HTTP/1.1" 200 -


Received request at /


INFO:werkzeug:127.0.0.1 - - [23/May/2025 14:56:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/May/2025 14:56:21] "POST / HTTP/1.1" 200 -


Received request at /
Processing POST request
Prediction completed: REAL


INFO:werkzeug:127.0.0.1 - - [23/May/2025 14:56:51] "POST / HTTP/1.1" 200 -


Received request at /
Processing POST request
Prediction completed: FAKE
